# Выбор начальных условий
а. Набор данных для классификации
Скачиваем датасет.
https://www.kaggle.com/datasets/marquis03/vehicle-classification  
  
В нем разные машины, смыл определять конкретную машину. Так как сам водитель, то   интересно сделать классификатор для камер наблюдения. В Москве есть система умный   город, можно таким способом обнаружить какие машины проезжали и найти преступника.  

b. Выбор метрик
Так как мы классифицируем, то есть классы, есть предсказания.  
Поэтому как в прошлых лабораторных использую f1-score, precision, accuracy, recall  

Работу выполнил Овчинников Дмитрий Максимович М8О-406Б-21  

In [ ]:
# Качаем датасет
import kagglehub

# Download latest version
path = kagglehub.dataset_download("marquis03/vehicle-classification")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/vehicle-classification


In [ ]:
# находим где он
!ls /root/.cache/kagglehub/datasets/marquis03/vehicle-classification/versions/1

test  train  val


In [ ]:
# полезный пак либ
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from torchvision.models import resnet18, ResNet18_Weights, vit_b_16, ViT_B_16_Weights
from sklearn.metrics import classification_report
import seaborn as sns
from copy import deepcopy

In [ ]:
# нашли где лежит
path_train = '/root/.cache/kagglehub/datasets/marquis03/vehicle-classification/versions/1/train'
path_test = '/root/.cache/kagglehub/datasets/marquis03/vehicle-classification/versions/1/val'

In [ ]:
# Нужно, чтобы был размер работы с моделью. Нужен перевод картинки в массив чисел (тензор) и нормализация, чтобы примерно одинаково влияло
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5] * 3, std=[0.5] * 3)
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5] * 3, std=[0.5] * 3)
])

In [ ]:
# Делал локально, можно менять выше
NUM_COUNT = 100

full_train_dataset = datasets.ImageFolder(root=path_train, transform=train_transforms)
full_test_dataset = datasets.ImageFolder(root=path_test, transform=test_transforms)

# Не нагружать карту всем датасетом
def get_subset(dataset, count):
    if count is None or count >= len(dataset):
        return dataset
    inx = random.sample(range(len(dataset)), count)

    return Subset(dataset, inx)

train_dataset = get_subset(full_train_dataset, NUM_COUNT)
test_dataset = get_subset(full_test_dataset, NUM_COUNT)

# Итераторы для данных
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Что вообще в классах
class_names = full_train_dataset.classes
print("Классы:", class_names)

Классы: ['SUV', 'bus', 'family sedan', 'fire engine', 'heavy truck', 'jeep', 'minibus', 'racing car', 'taxi', 'truck']


In [ ]:
# на видеокарте быстрее, но локально только 4 гига есть :C
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

def train_classifier(model, criterion, optimizer, train_loader, test_loader, num_epochs=3):
    model.to(device)
    # Учим
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            r  += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)

        print(f"Эпоха {epoch+1}/{num_epochs}, loss: {epoch_loss:.4f}")
    # Проверяем насколько круто. (предварительные данные)
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    acc = correct / total

    print(f"Насколько круто: {acc:.2f} {correct}/{total}")

cuda


---

# 2. Создание бейзлайна и оценка качества  
a. Обучить сверточную модель

In [ ]:
# Стандартные результаты
weights = ResNet18_Weights.DEFAULT
resnet18_m = resnet18(weights=weights)
resnet18_m.fc = nn.Linear(resnet18_m.fc.in_features, len(class_names))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet18_m.parameters(), lr=1e-3)

train_classifier(resnet18_m, criterion, optimizer, train_loader, test_loader)

Эпоха 1/3, loss: 1.8700
Эпоха 2/3, loss: 0.2609
Эпоха 3/3, loss: 0.1169
Насколько круто: 0.35 35/100


In [ ]:
# Стандартные результаты для трансформерной
weights = ViT_B_16_Weights.DEFAULT
vit_m = vit_b_16(weights=weights)
vit_m.heads.head = nn.Linear(vit_m.heads.head.in_features, len(class_names))

criterion_vit = nn.CrossEntropyLoss()
# Нужен свой т.к свои параметры и в прошлом остались от прошлой модели измененные
optimizer_vit = optim.Adam(vit_m.parameters(), lr=1e-4)

train_classifier(vit_m, criterion_vit, optimizer_vit, train_loader, test_loader)

Эпоха 1/3, loss: 2.1613
Эпоха 2/3, loss: 0.9563
Эпоха 3/3, loss: 0.4268
Насколько круто: 0.78 78/100


In [ ]:
# круто, но надо, чтобы более подробная картина была
def evalModel(model, test_data):
    model.eval()
    true_pred, pred = [], []

    with torch.no_grad():
        for inputs, labels in test_data:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            true_pred.extend(labels.cpu().numpy())
            pred.extend(preds.cpu().numpy())
    return true_pred, pred

def slovaric_info(model, test_data, class_names):
    true_pred_arr, pred_arr = evalModel(model, test_data)

    label_ids = list(range(len(class_names)))
    report = classification_report(
        true_pred_arr,
        pred_arr,
        labels=label_ids,
        target_names=class_names,
        zero_division=0,
        output_dict=True
    )

    return report


In [ ]:
# Извлекаем нужные метрики

def metrics_model(model, name, loader):
    print(f"for model {name}")
    info = slovaric_info(model, loader, class_names)
    f1 = info["macro avg"]["f1-score"]
    print(f"f1: {f1:.2f}")
    presicion = info["macro avg"]["precision"]
    print(f"presicion: {presicion:.2f}")
    acc = info["accuracy"]
    print(f"acc: {acc:.2f}")
    recall = info["macro avg"]["recall"]
    print(f"recall {recall:.2f}")
    return info


In [ ]:
# метрики для resnet18_m
metrics_model(resnet18_m,"resnet", test_loader)

for model resnet
f1: 0.26
presicion: 0.32
acc: 0.35
recall 0.31


{'SUV': {'precision': 0.2558139534883721,
  'recall': 1.0,
  'f1-score': 0.4074074074074074,
  'support': 11.0},
 'bus': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 10.0},
 'family sedan': {'precision': 0.5,
  'recall': 0.2727272727272727,
  'f1-score': 0.35294117647058826,
  'support': 11.0},
 'fire engine': {'precision': 0.8571428571428571,
  'recall': 0.5454545454545454,
  'f1-score': 0.6666666666666666,
  'support': 11.0},
 'heavy truck': {'precision': 0.6,
  'recall': 0.2727272727272727,
  'f1-score': 0.375,
  'support': 11.0},
 'jeep': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 8.0},
 'minibus': {'precision': 0.6666666666666666,
  'recall': 0.25,
  'f1-score': 0.36363636363636365,
  'support': 8.0},
 'racing car': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 11.0},
 'taxi': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 6.0},
 'truck': {'precision': 0.2777777777777778,
  'recall': 0.7692307692307693,
  'f1

In [ ]:
# метрики для vit_m
metrics_model(vit_m,"resnet", test_loader)

for model resnet
f1: 0.80
presicion: 0.86
acc: 0.78
recall 0.80


{'SUV': {'precision': 0.8333333333333334,
  'recall': 0.45454545454545453,
  'f1-score': 0.5882352941176471,
  'support': 11.0},
 'bus': {'precision': 1.0,
  'recall': 0.8,
  'f1-score': 0.8888888888888888,
  'support': 10.0},
 'family sedan': {'precision': 0.6,
  'recall': 0.8181818181818182,
  'f1-score': 0.6923076923076923,
  'support': 11.0},
 'fire engine': {'precision': 0.7333333333333333,
  'recall': 1.0,
  'f1-score': 0.8461538461538461,
  'support': 11.0},
 'heavy truck': {'precision': 1.0,
  'recall': 0.6363636363636364,
  'f1-score': 0.7777777777777778,
  'support': 11.0},
 'jeep': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 8.0},
 'minibus': {'precision': 0.3888888888888889,
  'recall': 0.875,
  'f1-score': 0.5384615384615384,
  'support': 8.0},
 'racing car': {'precision': 1.0,
  'recall': 0.9090909090909091,
  'f1-score': 0.9523809523809523,
  'support': 11.0},
 'taxi': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 6.0},
 'truck': {'preci

Vit будет покруче

---

# 3. улучшить бейзлайн

In [ ]:
# Улучшаем бейзлайн

# гипотеза
# Если машины не выровнены, то добавить повернутые фото и которые ездят в разные стороны может сделать лучше результаты
better_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5] * 3, std=[0.5] * 3)
])

In [ ]:

# обновляем наш датасет
full_train_better_transform = datasets.ImageFolder(root=path_train, transform=better_transform)
full_test_dataset = datasets.ImageFolder(root=path_test, transform=test_transforms)

train_dataset_extra = get_subset(full_train_better_transform, NUM_COUNT)
test_dataset = get_subset(full_test_dataset, NUM_COUNT)

# Итераторы для данных
train_loader = DataLoader(train_dataset_extra, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Проверка: что классы есть
class_names = full_train_better_transform.classes
print("Классы:", class_names)

Классы: ['SUV', 'bus', 'family sedan', 'fire engine', 'heavy truck', 'jeep', 'minibus', 'racing car', 'taxi', 'truck']


In [ ]:
# обучаем опять
weights = ResNet18_Weights.DEFAULT
resnet18_m_extra = resnet18(weights=weights)
resnet18_m_extra.fc = nn.Linear(resnet18_m_extra.fc.in_features, len(class_names))

criterion_extra = nn.CrossEntropyLoss()
optimizer_extra = optim.Adam(resnet18_m_extra.parameters(), lr=1e-3)

train_classifier(resnet18_m_extra, criterion_extra, optimizer_extra, train_loader, test_loader)

Эпоха 1/3, loss: 1.8512
Эпоха 2/3, loss: 0.5436
Эпоха 3/3, loss: 0.1699
Насколько круто: 0.34 34/100


In [ ]:
# лучше не стало, мб дело в мелкой выборке, либо таких мало в тест данных повернутых
metrics_model(resnet18_m_extra,"resnet extra", test_loader)

for model resnet extra
f1: 0.30
presicion: 0.49
acc: 0.34
recall 0.34


{'SUV': {'precision': 0.4, 'recall': 0.4, 'f1-score': 0.4, 'support': 10.0},
 'bus': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 9.0},
 'family sedan': {'precision': 1.0,
  'recall': 0.15384615384615385,
  'f1-score': 0.26666666666666666,
  'support': 13.0},
 'fire engine': {'precision': 0.375,
  'recall': 0.3,
  'f1-score': 0.3333333333333333,
  'support': 10.0},
 'heavy truck': {'precision': 0.16666666666666666,
  'recall': 0.625,
  'f1-score': 0.2631578947368421,
  'support': 8.0},
 'jeep': {'precision': 1.0,
  'recall': 0.08333333333333333,
  'f1-score': 0.15384615384615385,
  'support': 12.0},
 'minibus': {'precision': 0.6666666666666666,
  'recall': 0.6,
  'f1-score': 0.631578947368421,
  'support': 10.0},
 'racing car': {'precision': 1.0,
  'recall': 0.4,
  'f1-score': 0.5714285714285714,
  'support': 10.0},
 'taxi': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 7.0},
 'truck': {'precision': 0.2647058823529412,
  'recall': 0.8181818181818182,
  

In [ ]:
# А вот со второй не плохо
weights = ViT_B_16_Weights.DEFAULT
vit_m_extra = vit_b_16(weights=weights)
vit_m_extra.heads.head = nn.Linear(vit_m_extra.heads.head.in_features, len(class_names))

criterion_vit = nn.CrossEntropyLoss()
# Нужен свой т.к свои параметры и в прошлом остались от прошлой модели измененные
optimizer_vit = optim.Adam(vit_m_extra.parameters(), lr=1e-4)

train_classifier(vit_m_extra, criterion_vit, optimizer_vit, train_loader, test_loader)

Эпоха 1/3, loss: 2.1311
Эпоха 2/3, loss: 1.1097
Эпоха 3/3, loss: 0.5155
Насколько круто: 0.75 75/100


In [ ]:
# но опять же стало не сильно круче
metrics_model(vit_m_extra,"vit extra", test_loader)

for model vit extra
f1: 0.71
presicion: 0.77
acc: 0.75
recall 0.74


{'SUV': {'precision': 0.37037037037037035,
  'recall': 1.0,
  'f1-score': 0.5405405405405406,
  'support': 10.0},
 'bus': {'precision': 0.9,
  'recall': 1.0,
  'f1-score': 0.9473684210526315,
  'support': 9.0},
 'family sedan': {'precision': 0.8333333333333334,
  'recall': 0.7692307692307693,
  'f1-score': 0.8,
  'support': 13.0},
 'fire engine': {'precision': 1.0,
  'recall': 0.7,
  'f1-score': 0.8235294117647058,
  'support': 10.0},
 'heavy truck': {'precision': 0.8888888888888888,
  'recall': 1.0,
  'f1-score': 0.9411764705882353,
  'support': 8.0},
 'jeep': {'precision': 1.0,
  'recall': 0.16666666666666666,
  'f1-score': 0.2857142857142857,
  'support': 12.0},
 'minibus': {'precision': 1.0,
  'recall': 0.9,
  'f1-score': 0.9473684210526315,
  'support': 10.0},
 'racing car': {'precision': 0.8333333333333334,
  'recall': 1.0,
  'f1-score': 0.9090909090909091,
  'support': 10.0},
 'taxi': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 7.0},
 'truck': {'precision': 0.8

In [ ]:
# Гипотеза провалена 
# Возвращаем старый даталоадер
# 2 блока вернуть старый датасет

In [ ]:
NUM_COUNT = 100

full_train_dataset = datasets.ImageFolder(root=path_train, transform=train_transforms)
full_test_dataset = datasets.ImageFolder(root=path_test, transform=test_transforms)

# Не нагружать карту всем датасетом
def get_subset(dataset, count):
    if count is None or count >= len(dataset):
        return dataset
    inx = random.sample(range(len(dataset)), count)

    return Subset(dataset, inx)

train_dataset = get_subset(full_train_dataset, NUM_COUNT)
test_dataset = get_subset(full_test_dataset, NUM_COUNT)

# Итераторы для данных
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Что вообще вышло
class_names = full_train_dataset.classes
print("Классы:", class_names)

Классы: ['SUV', 'bus', 'family sedan', 'fire engine', 'heavy truck', 'jeep', 'minibus', 'racing car', 'taxi', 'truck']


In [ ]:
# Извлекаем нужные метрики

def metrics_model(model, name, loader):
    print(f"for model {name}")
    info = slovaric_info(model, loader, class_names)
    f1 = info["macro avg"]["f1-score"]
    print(f"f1: {f1:.2f}")
    presicion = info["macro avg"]["precision"]
    print(f"presicion: {presicion:.2f}")
    acc = info["accuracy"]
    print(f"acc: {acc:.2f}")
    recall = info["macro avg"]["recall"]
    print(f"recall {recall:.2f}")
    return info

гипотеза 2 - меняем гиперпараметры

In [ ]:
# Разная learning rate, и количество эпох
lr_new = [1e-3, 1e-4]
epoch_new = [3, 5]

res_info = []
batch = 6

for lr in lr_new:
    for epochs in epoch_new:
        print(f"resnet | Смотрим с : LR={lr}, Batch={batch}, Epochs={epochs}")
        train_loader_hp = DataLoader(train_dataset, batch_size=batch, shuffle=True, num_workers=0)
        test_loader_hp = DataLoader(test_dataset, batch_size=batch, shuffle=False)
        model_r = resnet18(weights=ResNet18_Weights.DEFAULT)
        model_r.fc = nn.Linear(model_r.fc.in_features, len(class_names))
        model_r = model_r.to(device)
        optimizer_r = optim.Adam(model_r.parameters(), lr=lr)
        criterion_r = nn.CrossEntropyLoss()
        train_classifier(model_r, criterion_r, optimizer_r, train_loader_hp, test_loader_hp, num_epochs=epochs)
        report_r = metrics_model(model_r,"resnet", test_loader_hp)
        print("")
        res_info.append({
            "lr": lr, "epochs": epochs,
            "f1": report_r["macro avg"]["f1-score"],
            "acc": report_r["accuracy"],
            "report": deepcopy(report_r)
        })

resnet | Смотрим с : LR=0.001, Batch=6, Epochs=3
Эпоха 1/3, loss: 2.3605
Эпоха 2/3, loss: 1.8121
Эпоха 3/3, loss: 1.0397
Насколько круто: 0.41 41/100
for model resnet
f1: 0.40
presicion: 0.46
acc: 0.41
recall 0.42

resnet | Смотрим с : LR=0.001, Batch=6, Epochs=5
Эпоха 1/5, loss: 2.3291
Эпоха 2/5, loss: 0.9094
Эпоха 3/5, loss: 0.8224
Эпоха 4/5, loss: 0.3504
Эпоха 5/5, loss: 0.4249
Насколько круто: 0.40 40/100
for model resnet
f1: 0.30
presicion: 0.41
acc: 0.40
recall 0.36

resnet | Смотрим с : LR=0.0001, Batch=6, Epochs=3
Эпоха 1/3, loss: 2.0527
Эпоха 2/3, loss: 0.7413
Эпоха 3/3, loss: 0.2947
Насколько круто: 0.79 79/100
for model resnet
f1: 0.78
presicion: 0.81
acc: 0.79
recall 0.79

resnet | Смотрим с : LR=0.0001, Batch=6, Epochs=5
Эпоха 1/5, loss: 2.0396
Эпоха 2/5, loss: 0.5930
Эпоха 3/5, loss: 0.2509
Эпоха 4/5, loss: 0.1114
Эпоха 5/5, loss: 0.0728
Насколько круто: 0.82 82/100
for model resnet
f1: 0.82
presicion: 0.85
acc: 0.82
recall 0.82



Стало по круче с меньше lr, больше эпох

In [ ]:
# Проверям аналогично
vit_info = []
for lr in lr_new:
    for epochs in epoch_new:
        print(f"vit | Смотрим с : LR={lr}, Epochs={epochs}")
        train_loader_hp = DataLoader(train_dataset, batch_size=batch, shuffle=True, num_workers=0)
        test_loader_hp = DataLoader(test_dataset, batch_size=batch, shuffle=False)
        model_v = vit_b_16(weights=ViT_B_16_Weights.DEFAULT)
        model_v.heads.head = nn.Linear(model_v.heads.head.in_features, len(class_names))
        model_v = model_v.to(device)
        optimizer_v = optim.Adam(model_v.parameters(), lr=lr)
        criterion_v = nn.CrossEntropyLoss()
        train_classifier(model_v, criterion_v, optimizer_v, train_loader_hp, test_loader_hp, num_epochs=epochs)
        report_v = metrics_model(model_v, "vit", test_loader_hp)
        print("")
        vit_info.append({
            "lr": lr, "epochs": epochs,
            "f1": report_v["macro avg"]["f1-score"],
            "acc": report_v["accuracy"],
            "report": deepcopy(report_v)
        })

vit | Смотрим с : LR=0.001, Epochs=3
Эпоха 1/3, loss: 2.7883
Эпоха 2/3, loss: 2.4403
Эпоха 3/3, loss: 2.4584
Насколько круто: 0.08 8/100
for model vit
f1: 0.03
presicion: 0.11
acc: 0.08
recall 0.11

vit | Смотрим с : LR=0.001, Epochs=5
Эпоха 1/5, loss: 2.9403
Эпоха 2/5, loss: 2.4109
Эпоха 3/5, loss: 2.3842
Эпоха 4/5, loss: 2.4118
Эпоха 5/5, loss: 2.4083
Насколько круто: 0.09 9/100
for model vit
f1: 0.03
presicion: 0.02
acc: 0.09
recall 0.12

vit | Смотрим с : LR=0.0001, Epochs=3
Эпоха 1/3, loss: 2.1851
Эпоха 2/3, loss: 0.6146
Эпоха 3/3, loss: 0.0966
Насколько круто: 0.86 86/100
for model vit
f1: 0.86
presicion: 0.87
acc: 0.86
recall 0.87

vit | Смотрим с : LR=0.0001, Epochs=5
Эпоха 1/5, loss: 1.9808
Эпоха 2/5, loss: 0.5393
Эпоха 3/5, loss: 0.1245
Эпоха 4/5, loss: 0.0474
Эпоха 5/5, loss: 0.0163
Насколько круто: 0.92 92/100
for model vit
f1: 0.91
presicion: 0.92
acc: 0.92
recall 0.92



Стало еще лучше! Гипотеза работает

Другая архитектура

In [ ]:
# Меняем модель
from torchvision.models import mobilenet_v3_small, MobileNet_V3_Small_Weights
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights


model_mobile = mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.DEFAULT)
model_mobile.classifier[3] = nn.Linear(model_mobile.classifier[3].in_features, len(class_names))
model_mobile = model_mobile.to(device)

optimizer_mobile = optim.Adam(model_mobile.parameters(), lr=1e-4)
criterion_mobile = nn.CrossEntropyLoss()

train_classifier(model_mobile, criterion_mobile, optimizer_mobile, train_loader_hp, test_loader_hp, num_epochs=3)

report_mobile = metrics_model(model_mobile, "mobile_net", test_loader_hp)

Эпоха 1/3, loss: 2.2865
Эпоха 2/3, loss: 2.0129
Эпоха 3/3, loss: 1.7536
Насколько круто: 0.33 33/100
for model mobile_net
f1: 0.31
presicion: 0.47
acc: 0.33
recall 0.36


In [ ]:
# Меняем модель
model_effnet = efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)
model_effnet.classifier[1] = nn.Linear(model_effnet.classifier[1].in_features, len(class_names))
model_effnet = model_effnet.to(device)

optimizer_eff = optim.Adam(model_effnet.parameters(), lr=1e-4)
criterion_eff = nn.CrossEntropyLoss()

train_classifier(model_effnet, criterion_eff, optimizer_eff, train_loader_hp, test_loader_hp, num_epochs=3)

report_eff = metrics_model(model_effnet, "efficient_net", test_loader_hp)

Эпоха 1/3, loss: 2.2897
Эпоха 2/3, loss: 2.0194
Эпоха 3/3, loss: 1.7843
Насколько круто: 0.49 49/100
for model efficient_net
f1: 0.49
presicion: 0.62
acc: 0.49
recall 0.51


Не работает гипотеза
Самое крутое что получили это другие параметры - больше эпох и лучше lr

---

Улучшенный бейслайн

In [ ]:
def metrics_model(model, name, loader):
    print(f"for model {name}")
    info = slovaric_info(model, loader, class_names)
    f1 = info["macro avg"]["f1-score"]
    print(f"f1: {f1:.2f}")
    presicion = info["macro avg"]["precision"]
    print(f"presicion: {presicion:.2f}")
    acc = info["accuracy"]
    print(f"acc: {acc:.2f}")
    recall = info["macro avg"]["recall"]
    print(f"recall {recall:.2f}")
    return info


Обучаем на лучших параметрах, другие не отработали гипотезы

In [ ]:
weights = ResNet18_Weights.DEFAULT
resnet18_m = resnet18(weights=weights)
resnet18_m.fc = nn.Linear(resnet18_m.fc.in_features, len(class_names))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet18_m.parameters(), lr=1e-4)

train_classifier(resnet18_m, criterion, optimizer, train_loader, test_loader, num_epochs=5)
metrics_model(resnet18_m,"resnet18 ", test_loader)

Эпоха 1/5, loss: 2.2194
Эпоха 2/5, loss: 0.9223
Эпоха 3/5, loss: 0.4549
Эпоха 4/5, loss: 0.2809
Эпоха 5/5, loss: 0.1698
Насколько круто: 0.72 72/100
for model resnet18 
f1: 0.73
presicion: 0.80
acc: 0.72
recall 0.72


{'SUV': {'precision': 0.9,
  'recall': 0.8181818181818182,
  'f1-score': 0.8571428571428571,
  'support': 11.0},
 'bus': {'precision': 1.0,
  'recall': 0.6666666666666666,
  'f1-score': 0.8,
  'support': 9.0},
 'family sedan': {'precision': 0.35294117647058826,
  'recall': 0.8571428571428571,
  'f1-score': 0.5,
  'support': 7.0},
 'fire engine': {'precision': 0.3333333333333333,
  'recall': 0.7142857142857143,
  'f1-score': 0.45454545454545453,
  'support': 7.0},
 'heavy truck': {'precision': 0.7,
  'recall': 0.7,
  'f1-score': 0.7,
  'support': 10.0},
 'jeep': {'precision': 1.0,
  'recall': 0.7,
  'f1-score': 0.8235294117647058,
  'support': 10.0},
 'minibus': {'precision': 1.0,
  'recall': 0.6923076923076923,
  'f1-score': 0.8181818181818182,
  'support': 13.0},
 'racing car': {'precision': 1.0,
  'recall': 0.7272727272727273,
  'f1-score': 0.8421052631578947,
  'support': 11.0},
 'taxi': {'precision': 0.8333333333333334,
  'recall': 0.5,
  'f1-score': 0.625,
  'support': 10.0},
 'tr

In [ ]:
weights = ViT_B_16_Weights.DEFAULT
vit_m = vit_b_16(weights=weights)
vit_m.heads.head = nn.Linear(vit_m.heads.head.in_features, len(class_names))

criterion_vit = nn.CrossEntropyLoss()
# Нужен свой т.к свои параметры и в прошлом остались от прошлой модели измененные
optimizer_vit = optim.Adam(vit_m.parameters(), lr=1e-4)

train_classifier(vit_m, criterion_vit, optimizer_vit, train_loader, test_loader, num_epochs=5)
metrics_model(vit_m,"vit", test_loader)

Эпоха 1/5, loss: 2.1143
Эпоха 2/5, loss: 0.9318
Эпоха 3/5, loss: 0.3987
Эпоха 4/5, loss: 0.1848
Эпоха 5/5, loss: 0.0924
Насколько круто: 0.83 83/100
for model vit
f1: 0.82
presicion: 0.88
acc: 0.83
recall 0.82


{'SUV': {'precision': 0.625,
  'recall': 0.9090909090909091,
  'f1-score': 0.7407407407407407,
  'support': 11.0},
 'bus': {'precision': 1.0,
  'recall': 0.8888888888888888,
  'f1-score': 0.9411764705882353,
  'support': 9.0},
 'family sedan': {'precision': 1.0,
  'recall': 0.42857142857142855,
  'f1-score': 0.6,
  'support': 7.0},
 'fire engine': {'precision': 0.7777777777777778,
  'recall': 1.0,
  'f1-score': 0.875,
  'support': 7.0},
 'heavy truck': {'precision': 1.0,
  'recall': 0.6,
  'f1-score': 0.75,
  'support': 10.0},
 'jeep': {'precision': 1.0,
  'recall': 0.9,
  'f1-score': 0.9473684210526315,
  'support': 10.0},
 'minibus': {'precision': 1.0,
  'recall': 0.8461538461538461,
  'f1-score': 0.9166666666666666,
  'support': 13.0},
 'racing car': {'precision': 1.0,
  'recall': 0.9090909090909091,
  'f1-score': 0.9523809523809523,
  'support': 11.0},
 'taxi': {'precision': 0.625,
  'recall': 1.0,
  'f1-score': 0.7692307692307693,
  'support': 10.0},
 'truck': {'precision': 0.75,


---


# 4 Собственная имплементация

In [ ]:
# кастомная имплмементация 
class CustomCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CustomCNN, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 32, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(128, 256, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(256, 512, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(512 * 7 * 7, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.features(x)
        x = self.flatten(x)
        x = self.dropout(x)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model_custom = CustomCNN(num_classes=10).to(device)
print(model_custom)

CustomCNN(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU(inplace=True)
    (6): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (13): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15)

In [ ]:
# Работает отвратительно :C
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_custom.parameters(), lr=1e-3)

train_classifier(model_custom, criterion, optimizer, train_loader, test_loader, num_epochs=3)
metrics_model(model_custom,"cnn custom ", test_loader)

Эпоха 1/3, loss: 4.4152
Эпоха 2/3, loss: 2.3111
Эпоха 3/3, loss: 2.2997
Насколько круто: 0.13 13/100
for model cnn custom 
f1: 0.02
presicion: 0.01
acc: 0.13
recall 0.10


{'SUV': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 11.0},
 'bus': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 9.0},
 'family sedan': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 7.0},
 'fire engine': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 7.0},
 'heavy truck': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 10.0},
 'jeep': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 10.0},
 'minibus': {'precision': 0.13,
  'recall': 1.0,
  'f1-score': 0.23008849557522124,
  'support': 13.0},
 'racing car': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 11.0},
 'taxi': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 10.0},
 'truck': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 12.0},
 'accuracy': 0.13,
 'macro avg': {'precision': 0.013000000000000001,
  'recall': 0.1,
  'f1-score': 0.023008849557522124,
  'support': 100.0},
 'w

In [ ]:
# кастомная имплмементация 
class CustomTransformer(nn.Module):
    def __init__(self, image_size=224,
                 patch_size=16,
                 in_channels=3,
                 num_classes=10,
                 embed_dim=256,
                 num_heads=8,
                 depth=8,
                 dropout=0.1):

        super(CustomTransformer, self).__init__()
        assert image_size % patch_size == 0
        self.num_patches = (image_size // patch_size) ** 2

        self.patch_embed = nn.Conv2d(in_channels, embed_dim,
                                    kernel_size=patch_size,
                                    stride=patch_size)

        self.cls_token = nn.Parameter(torch.randn(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(
            torch.randn(1, self.num_patches + 1, embed_dim)
        )

        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=embed_dim,
                nhead=num_heads,
                dropout=dropout,
                activation='gelu',
                batch_first=True
            ),
            num_layers=depth
        )

        self.mlp_head = nn.Sequential(
            nn.LayerNorm(embed_dim),
            nn.Linear(embed_dim, num_classes))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, C, H, W = x.shape

        x = self.patch_embed(x) 
        x = x.flatten(2).transpose(1, 2)

        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)

        x += self.pos_embed
        x = self.dropout(x)

        x = self.transformer(x)

        x = x[:, 0]
        return self.mlp_head(x)

model_transformer = CustomTransformer(
    image_size=224,
    patch_size=16,
    num_classes=10,
    embed_dim=256,
    num_heads=8,
    depth=8,
    dropout=0.1
).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_transformer.parameters(), lr=1e-3)

train_classifier(model_transformer, criterion, optimizer, train_loader, test_loader, num_epochs=3)
metrics_model(model_transformer,"transformer custom ", test_loader)

Эпоха 1/3, loss: 2.9990
Эпоха 2/3, loss: 2.8185
Эпоха 3/3, loss: 2.5807
Насколько круто: 0.10 10/100
for model transformer custom 
f1: 0.02
presicion: 0.01
acc: 0.10
recall 0.10


{'SUV': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 11.0},
 'bus': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 9.0},
 'family sedan': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 7.0},
 'fire engine': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 7.0},
 'heavy truck': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 10.0},
 'jeep': {'precision': 0.1,
  'recall': 1.0,
  'f1-score': 0.18181818181818182,
  'support': 10.0},
 'minibus': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 13.0},
 'racing car': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 11.0},
 'taxi': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 10.0},
 'truck': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 12.0},
 'accuracy': 0.1,
 'macro avg': {'precision': 0.01,
  'recall': 0.1,
  'f1-score': 0.01818181818181818,
  'support': 100.0},
 'weighted avg':

In [ ]:
# Улучшаем бейслайн
model_custom = CustomCNN(num_classes=10).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_custom.parameters(), lr=1e-5)

train_classifier(model_custom, criterion, optimizer, train_loader, test_loader, num_epochs=5)
metrics_model(model_custom,"cnn custom ", test_loader)

Эпоха 1/5, loss: 2.3043
Эпоха 2/5, loss: 2.2774
Эпоха 3/5, loss: 2.2721
Эпоха 4/5, loss: 2.2568
Эпоха 5/5, loss: 2.2412
Насколько круто: 0.07 7/100
for model cnn custom 
f1: 0.01
presicion: 0.01
acc: 0.07
recall 0.10


{'SUV': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 11.0},
 'bus': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 9.0},
 'family sedan': {'precision': 0.07,
  'recall': 1.0,
  'f1-score': 0.1308411214953271,
  'support': 7.0},
 'fire engine': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 7.0},
 'heavy truck': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 10.0},
 'jeep': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 10.0},
 'minibus': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 13.0},
 'racing car': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 11.0},
 'taxi': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 10.0},
 'truck': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 12.0},
 'accuracy': 0.07,
 'macro avg': {'precision': 0.007000000000000001,
  'recall': 0.1,
  'f1-score': 0.01308411214953271,
  'support': 100.0},
 'wei

In [ ]:
model_transformer = CustomTransformer(
    image_size=224,    # Стандартный размер для большинства датасетов
    patch_size=16,     # 224/16 = 14 -> 14x14 патчей
    num_classes=10,    # Количество классов
    embed_dim=256,
    num_heads=8,
    depth=8,
    dropout=0.1
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_transformer.parameters(), lr=1e-5)

train_classifier(model_transformer, criterion, optimizer, train_loader, test_loader, num_epochs=5)
metrics_model(model_transformer,"transformer custom ", test_loader)

Эпоха 1/5, loss: 2.3523
Эпоха 2/5, loss: 2.2862
Эпоха 3/5, loss: 2.2436
Эпоха 4/5, loss: 2.2377
Эпоха 5/5, loss: 2.1934
Насколько круто: 0.10 10/100
for model transformer custom 
f1: 0.04
presicion: 0.02
acc: 0.10
recall 0.14


{'SUV': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 11.0},
 'bus': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 9.0},
 'family sedan': {'precision': 0.06557377049180328,
  'recall': 0.5714285714285714,
  'f1-score': 0.11764705882352941,
  'support': 7.0},
 'fire engine': {'precision': 0.15384615384615385,
  'recall': 0.8571428571428571,
  'f1-score': 0.2608695652173913,
  'support': 7.0},
 'heavy truck': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 10.0},
 'jeep': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 10.0},
 'minibus': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 13.0},
 'racing car': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 11.0},
 'taxi': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 10.0},
 'truck': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 12.0},
 'accuracy': 0.1,
 'macro avg': {'precision': 0.021941992433795715,
  

Как итог ясно: что модели делали умные люди и нормально не получилось обучить, хоть и старался. :C

| Модель | **f1** | **precision** | **accuracy**| **recall**|
| --------| :---------: | :---------: |------- | -------|
| resnet18 |  0.26    |    0.32    | 0.35| 0.31|
| vit   |  0.80|  0.86  | 0.78| 0.80|
| resnet18 (разные картинки изменение)                     |    0.30    |    0.49    |0.34 | 0.34|
| vit (разные картинки изменение) |    0.71    |    0.77    |0.75 | 0.74|
| resnet18 (лучшие параметры)                     |   0.83   |    0.85    |0.82 | 0.82|
| vit (лучшие параметры) |    0.91    |    0.92    |0.92 | 0.92|
| собственный (CNN)                     |   0.02   |    0.01    |0.13 | 0.10|
| собственный (трансформер) |   0.02   |    0.01    |0.10 | 0.10|
| собственный (CNN) (улучшенный )                    |   0.01   |    0.01    |0.07 | 0.10|
| собственный (трансформер) (улучшенный) |   0.04   |    0.02    |0.10 | 0.14|

Вывод - люди сделали крутые модели, которые работают. Меня поразило, что можно так высоко в 92 процента делать распознавание. Теперь ясно, что такая система в Москве может работать для распознавания и это несомненно очень круто, что мы живем в такое время